# Chapter 05: 모델 로딩

## 1. 학습 목표

* gpt-oss-20b 모델을 효율적으로 로드하는 전략을 수립한다.
* 4-bit 양자화(QLoRA)를 적용하여 VRAM 사용량을 획기적으로 줄인다.
* 모델의 내부 구조를 분석하여 LoRA를 적용할 타겟 모듈을 식별한다.
* 학습을 위한 전처리(Gradient Checkpointing 등)를 수행한다.

## 2. 모델 로딩 전략

### 2.1 메모리 요구량 분석

20B(200억) 파라미터 모델을 로드하고 학습하려면 막대한 메모리가 필요하다. 정밀도(Precision)에 따른 요구량은 다음과 같다.

| 정밀도 | 파라미터당 용량 | 모델 크기(Load) | 학습 시 필요 VRAM (LoRA 기준) |
| --- | --- | --- | --- |
| **FP32** | 4 bytes | ~80 GB | ~100 GB+ (불가능) |
| **FP16/BF16** | 2 bytes | ~40 GB | ~48 GB (A100 필요) |
| **INT8** | 1 byte | ~20 GB | ~24 GB (RTX 3090/4090) |
| **INT4 (QLoRA)** | **0.5 byte** | **~10 GB** | **~14-16 GB (RTX 3090/4090 여유)** |

대부분의 개인 및 연구 환경에서는 **INT4(4-bit) 양자화**가 필수적이다. 이를 통해 모델 성능 저하를 최소화하면서 메모리 사용량을 1/4 수준으로 줄일 수 있다.

## 3. 4-bit 양자화 설정 (BitsAndBytes)

HuggingFace의 `bitsandbytes` 라이브러리를 사용하여 4-bit 로딩 설정을 구성한다. QLoRA 논문에서 제안한 **NF4(NormalFloat 4-bit)** 데이터 타입을 사용하여 정보 손실을 최소화한다.

In [5]:
import torch
from transformers import BitsAndBytesConfig

def get_bnb_config():
    """
    QLoRA 학습에 최적화된 BitsAndBytes 양자화 설정을 반환한다.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,                     # 4-bit 양자화 활성화
        bnb_4bit_quant_type="nf4",             # NormalFloat 4-bit (가중치 분포에 최적화)
        bnb_4bit_compute_dtype=torch.bfloat16, # 연산은 bfloat16으로 수행 (안정성 향상)
        bnb_4bit_use_double_quant=True,        # 이중 양자화 (양자화 상수도 양자화하여 추가 절약)
    )
    return bnb_config

print("4-bit 양자화 설정(NF4, Double Quantization)이 준비되었다.")

4-bit 양자화 설정(NF4, Double Quantization)이 준비되었다.


## 4. 모델 및 토크나이저 로드

### 4.1 일반 모델 로드
Hugging Face 허브의 일반적인 모델 로딩 시 사용한다. 여기서는 `Qwen/Qwen3-14B`를 불러온다.

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_name):
    """
    모델과 토크나이저를 로드하는 함수다.
    """
    print(f"모델 로딩 시작: {model_name}")

    # 1. 양자화 설정 가져오기
    bnb_config = get_bnb_config()

    # 2. 모델 로드
    # device_map="auto": 가용한 GPU에 자동으로 모델을 분배한다.
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="sdpa" # PyTorch 2.0+ Scaled Dot Product Attention 사용
    )

    # 3. 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )

    # 패딩 토큰 설정 (생성 모델의 경우 필수)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # LoRA 학습 시 패딩은 오른쪽이 아닌 쪽에 두는 것이 일반적이나,
    # 최신 라이브러리(trl)는 right padding도 잘 처리한다. 여기서는 right로 설정한다.
    tokenizer.padding_side = "right"

    print(f"로드 완료: {model.config.model_type}")
    print(f"메모리 사용량: {model.get_memory_footprint() / 1e9:.2f} GB")

    return model, tokenizer

# 실제 모델 로드 (시간이 소요될 수 있다)
model_id = "Qwen/Qwen3-14B" 
model, tokenizer = load_model_and_tokenizer(model_id)

모델 로딩 시작: Qwen/Qwen3-14B


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

로드 완료: qwen3
메모리 사용량: 9.72 GB


### 4.2. gpt-oss-20b: MXFP4 모델 로드
gpt-oss-20b의 경우 기본적으로 MXPF4를 지원한다. 이를 위하여 `BitsAndBytesConfig`를 빼고 `Mxfp4Config()`로 로드한다.

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Mxfp4Config

model_id = "openai/gpt-oss-20b"

quant_config = Mxfp4Config()  # MXFP4 유지

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype="auto",
    attn_implementation="eager",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
tokenizer.padding_side = "right"

`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## 5. 학습을 위한 모델 준비

로드된 4-bit 모델을 바로 학습할 수는 없다. `peft` 라이브러리의 전처리 함수를 통해 학습 가능한 상태로 만들어야 한다. 또한, 메모리 절약을 위해 **Gradient Checkpointing**을 활성화한다.

In [9]:
from peft import prepare_model_for_kbit_training

def prepare_model_for_training(model):
    """
    QLoRA 학습을 위해 모델을 전처리하는 함수다.
    """
    # 1. Gradient Checkpointing 활성화
    # 중간 활성화 값을 저장하지 않고 역전파 시 재계산하여 메모리를 절약한다.
    # 속도는 약 20% 느려지지만 VRAM을 30% 이상 아낄 수 있다.
    model.gradient_checkpointing_enable()

    # 2. k-bit 학습 준비
    # LayerNorm 등 안정성이 필요한 레이어를 fp32로 변환하고,
    # requires_grad 설정을 조정한다.
    model = prepare_model_for_kbit_training(
        model,
        use_gradient_checkpointing=True
    )

    print("모델이 학습 준비 상태(k-bit training ready)가 되었다.")
    return model

model = prepare_model_for_training(model)

모델이 학습 준비 상태(k-bit training ready)가 되었다.


## 6. 모델 구조 분석 및 LoRA 타겟 설정

LoRA를 효과적으로 적용하기 위해서는 어떤 레이어(모듈)에 어댑터를 부착할지 결정해야 한다. 일반적으로 Attention 관련 레이어(`q_proj`, `v_proj` 등)와 FFN 레이어(`up_proj`, `gate_proj` 등) 모두에 적용하면 성능이 가장 좋다.

In [10]:
def find_all_linear_names(model):
    """
    모델 내의 모든 선형 레이어(Linear) 이름을 찾아 LoRA 타겟으로 반환한다.
    출력 레이어(lm_head)는 제외한다.
    """
    cls = torch.nn.Linear
    # 4-bit 모델의 경우 LinearLayer가 아닌 bnb.nn.Linear4bit 클래스일 수 있다.
    import bitsandbytes as bnb
    if isinstance(model, bnb.nn.Linear4bit):
        cls = bnb.nn.Linear4bit

    lora_module_names = set()

    for name, module in model.named_modules():
        # 4-bit Linear 또는 일반 Linear 레이어 찾기
        if isinstance(module, (torch.nn.Linear, bnb.nn.Linear4bit, bnb.nn.Linear8bitLt)):
            names = name.split('.')
            lora_module_names.add(names[-1])

    # lm_head(출력층)는 보통 LoRA 적용 대상에서 제외한다.
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

target_modules = find_all_linear_names(model)

print(f"LoRA 적용 타겟 모듈: {target_modules}")

LoRA 적용 타겟 모듈: ['q_proj', 'o_proj', 'v_proj', 'k_proj']


## 7. 메모리 최적화 팁

`gpt-oss-20b`와 같은 큰 모델을 다룰 때 OOM(Out of Memory) 오류가 발생한다면 다음 팁을 적용한다.

1. **배치 크기 축소**: `per_device_train_batch_size`를 1로 설정하고 `gradient_accumulation_steps`를 늘려 실효 배치 크기를 맞춘다.
2. **시퀀스 길이 제한**: `max_seq_length`를 512 또는 1024로 줄인다. (기본 2048/4096은 메모리를 많이 차지함)
3. **Paged Optimizer 사용**: `paged_adamw_8bit` 옵티마이저는 GPU 메모리 부족 시 CPU RAM을 활용한다.
4. **캐시 정리**: 학습 루프 사이사이에 `torch.cuda.empty_cache()`를 호출한다.

## 8. 요약

이번 챕터에서는 `gpt-oss-20b`급 대형 모델을 소비자용 GPU에서도 학습할 수 있도록 **4-bit QLoRA 로딩** 과정을 실습했다.

1. **BitsAndBytesConfig**로 NF4 양자화와 이중 양자화를 설정했다.
2. `prepare_model_for_kbit_training`으로 모델을 학습 가능한 상태로 만들었다.
3. 모델 구조를 분석하여 모든 선형 레이어를 **LoRA 타겟**으로 식별했다.

이제 모델이 준비되었으므로, 본격적으로 데이터를 주입하여 학습을 시킬 차례다.

다음 챕터는 **Chapter 06: LoRA를 활용한 SFT**로, 준비된 모델과 데이터셋을 결합하여 실제 학습(Training) 루프를 돌리는 과정을 다룬다.